In [ ]:
import mysql.connector
from datetime import date

class DatabaseManager:
    def __init__(self):
        self.conn = mysql.connector.connect(host='localhost', database='task_management', user='root', password='')
        self.mycursor = self.conn.cursor()

    def __del__(self):
        self.conn.close()

class User(DatabaseManager):
    def __init__(self):
        super().__init__()

    def get_data(self):
        try:
            self.username = input("Enter the username: ")
            self.fullname = input("Enter the fullname: ")
            self.email = input("Enter the email: ")
            sql = "INSERT INTO user_info(username, name, email) VALUES (%s, %s, %s)"
            values = (self.username, self.fullname, self.email)
            self.mycursor.execute(sql, values)
            self.conn.commit()
            print("User added successfully.")
        except mysql.connector.Error as e:
            print(f"Error adding user: {e}")

    def update_profile(self):
        try:
            self.userid = int(input("Enter the user id to update the record of the user: "))
            self.mycursor.execute("SELECT * FROM user_info WHERE user_id = %s", (self.userid,))
            result = self.mycursor.fetchone()
            if not result:
                print('User does not exist')
            else:
                self.username = input("Enter the UserName for update: ")
                self.email = input("Enter the Email for update: ")
                update_sql = "UPDATE user_info SET username = %s, email = %s WHERE user_id = %s"
                update_data = (self.username, self.email, self.userid)
                self.mycursor.execute(update_sql, update_data)
                self.conn.commit()
                print("User profile updated successfully.")
        except mysql.connector.Error as e:
            print(f"Error updating user profile: {e}")

    def get_profile(self):
        try:
            self.mycursor.execute("SELECT * FROM user_info")
            results = self.mycursor.fetchall()
            if not results:
                print("No user profiles found.")
            else:
                for row in results:
                    user_id, username, name, email = row
                    print(user_id, username, name, email)
        except mysql.connector.Error as e:
            print(f"Error fetching user profiles: {e}")

class Task(DatabaseManager):
    def __init__(self):
        super().__init__()

    def add_task(self):
        try:
            self.userid = int(input("Enter the user id to assign the task for that user: "))
            self.mycursor.execute("SELECT * FROM user_info WHERE user_id = %s", (self.userid,))
            result = self.mycursor.fetchone()
            if not result:
                print('User does not exist')
            else:
                print("Task Details:")
                self.task_id = int(input("Enter the task id: "))
                self.title = input("Enter the task title: ")
                self.description = input("Enter the task description: ")
                self.category = input("Enter the category of the task: ")
                self.due_date = input("Enter the task due_date (YYYY-MM-DD): ")
                self.priority = input("Enter the task priority (eg. High, Medium, Low): ")
                self.status = input("Enter the Status of the task (eg. not started, in progress, completed): ")
                self.task_assigned_to_user_id = self.userid
                self.task_assigned_to_user_name = result[2]  # Fetch the user's name from the previous query
                self.assign_date = date.today()
                sql = "INSERT INTO task_info(task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date,due_date, priority, status) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s)"
                values = (self.task_assigned_to_user_id, self.task_assigned_to_user_name, self.task_id, self.title, self.description, self.category, self.assign_date, self.due_date, self.priority, self.status)
                self.mycursor.execute(sql, values)
                self.conn.commit()
                print("Task added successfully.")
        except mysql.connector.Error as e:
            print(f"Error adding task: {e}")

    def update_task(self):
        try:
            self.taskid = int(input("Enter the task id to update the record of the task: "))
            self.mycursor.execute("SELECT * FROM task_info WHERE task_id = %s", (self.taskid,))
            result = self.mycursor.fetchone()
            if not result:
                print('Task id not found in the record, please enter a valid task id: ')
            else:
                self.due_date = input("Enter the task due_date (YYYY-MM-DD): ")
                self.status = input("Enter the Status of the task (eg. not started, in progress, completed): ")
                update_sql = "UPDATE task_info SET due_date = %s, status = %s WHERE task_id = %s"
                update_data = (self.due_date, self.status, self.taskid)
                self.mycursor.execute(update_sql, update_data)
                self.conn.commit()
                print("Task updated successfully.")
        except mysql.connector.Error as e:
            print(f"Error updating task: {e}")

    def mark_as_completed(self):
        try:
            self.taskid = int(input("Enter the task id to mark as completed: "))
            self.mycursor.execute("SELECT * FROM task_info WHERE task_id = %s", (self.taskid,))
            result = self.mycursor.fetchone()
            if not result:
                print('Task id not found in the record, please enter a valid task id: ')
            else:
                if result[8].lower() == 'completed':
                    print("Task is already marked as Completed.")
                else:
                    update_sql = "UPDATE task_info SET status = 'completed' WHERE task_id = %s"
                    self.mycursor.execute(update_sql, (self.taskid,))
                    self.conn.commit()
                    print("Marked task as Completed.")
        except mysql.connector.Error as e:
            print(f"Error marking task as completed: {e}")

    def delete_task(self):
        try:
            self.taskid = int(input("Enter the task id to delete the record of the task: "))
            self.mycursor.execute("SELECT * FROM task_info WHERE task_id = %s", (self.taskid,))
            result = self.mycursor.fetchone()
            if not result:
                print('Task id not found in the record, please enter a valid task id: ')
            else:
                self.mycursor.execute("DELETE FROM task_info WHERE task_id = %s", (self.taskid,))
                self.conn.commit()
                print("Task deleted successfully.")
        except mysql.connector.Error as e:
            print(f"Error deleting task: {e}")

class TaskList(DatabaseManager):
    def __init__(self):
        super().__init__()

    def filtered_task(self):
        while True:
            try:
                choice = int(input("Enter the choice (1: status, 2: due_date, 3: priority, 4: exit): "))
                if choice == 1:
                    self.status = input("Filtered by status (eg. not started, in progress, completed): ")
                    self.mycursor.execute("SELECT * FROM task_info WHERE status = %s", (self.status,))
                    results = self.mycursor.fetchall()
                    if not results:
                        print("No tasks found with the selected status.")
                    else:
                        for row in results:
                            task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date, due_date, priority, status = row
                            print()
                            print("Filtered by status:")
                            print("User Id : ", task_assigned_to_user)
                            print("User Name that Assign the Task: ", name_of_user_toassign_the_task)
                            print("Task Id: ", task_id)
                            print("Task Title: ", title)
                            print("Task Description: ", description)
                            print("Task Category: ", category)
                            print("Task Assign Date: ", assign_date)
                            print("Task Due Date: ", due_date)
                            print("Task Priority: ", priority)
                            print("Task Status: ", status)
                            print()
                elif choice == 2:
                    self.due_date = input("Filtered by due_date (YYYY-MM-DD): ")
                    self.mycursor.execute("SELECT * FROM task_info WHERE due_date = %s", (self.due_date,))
                    results = self.mycursor.fetchall()
                    if not results:
                        print("No tasks found with the selected due date.")
                    else:
                        for row in results:
                            task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date, due_date, priority, status = row
                            print()
                            print("Filtered by due_date:")
                            print("User Id : ", task_assigned_to_user)
                            print("User Name that Assign the Task: ", name_of_user_toassign_the_task)
                            print("Task Id: ", task_id)
                            print("Task Title: ", title)
                            print("Task Description: ", description)
                            print("Task Category: ", category)
                            print("Task Assign Date: ", assign_date)
                            print("Task Due Date: ", due_date)
                            print("Task Priority: ", priority)
                            print("Task Status: ", status)
                            print()
                elif choice == 3:
                    self.priority = input("Filtered by priority (eg. High, Medium, Low): ")
                    self.mycursor.execute("SELECT * FROM task_info WHERE priority = %s", (self.priority,))
                    results = self.mycursor.fetchall()
                    if not results:
                        print("No tasks found with the selected priority.")
                    else:
                        for row in results:
                            task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date, due_date, priority, status = row
                            print()
                            print("Filtered by priority:")
                            print("User Id : ", task_assigned_to_user)
                            print("User Name that Assign the Task: ", name_of_user_toassign_the_task)
                            print("Task Id: ", task_id)
                            print("Task Title: ", title)
                            print("Task Description: ", description)
                            print("Task Category: ", category)
                            print("Task Assign Date: ", assign_date)
                            print("Task Due Date: ", due_date)
                            print("Task Priority: ", priority)
                            print("Task Status: ", status)
                            print()
                elif choice == 4:
                    break
                else:
                    print("Invalid choice!")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except mysql.connector.Error as e:
                print(f"Error fetching filtered tasks: {e}")

    def display_tasks(self):
        try:
            self.mycursor.execute("SELECT * FROM task_info")
            results = self.mycursor.fetchall()
            if not results:
                print("No tasks found.")
            else:
                for row in results:
                    task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date, due_date, priority, status = row
                    print()
                    print("User Id: ", task_assigned_to_user)
                    print("User Name that Assign the Task: ", name_of_user_toassign_the_task)
                    print("Task Id: ", task_id)
                    print("Task Title: ", title)
                    print("Task Description: ", description)
                    print("Task Category: ", category)
                    print("Task Assign Date: ", assign_date)
                    print("Task Due Date: ", due_date)
                    print("Task Priority: ", priority)
                    print("Task Status: ", status)
                    print()
        except mysql.connector.Error as e:
            print(f"Error fetching all tasks: {e}")

class Category(DatabaseManager):
    def __init__(self):
        super().__init__()
        self.name = input("Enter the category name: ")

    def view_task_by_category(self):
        try:
            self.mycursor.execute("SELECT * FROM task_info WHERE category = %s", (self.name,))
            results = self.mycursor.fetchall()
            if not results:
                print(f"No tasks found in the '{self.name}' category.")
            else:
                for row in results:
                    task_assigned_to_user, name_of_user_toassign_the_task, task_id, title, description, category, assign_date, due_date, priority, status = row
                    print()
                    print("User Id : ", task_assigned_to_user)
                    print("User Name that Assign the Task: ", name_of_user_toassign_the_task)
                    print("Task Id: ", task_id)
                    print("Task Title: ", title)
                    print("Task Description: ", description)
                    print("Task Category: ", category)
                    print("Task Assign Date: ", assign_date)
                    print("Task Due Date: ", due_date)
                    print("Task Priority: ", priority)
                    print("Task Status: ", status)
                    print()
        except mysql.connector.Error as e:
            print(f"Error viewing tasks by category: {e}")

class Analytics(DatabaseManager):
    def __init__(self):
        super().__init__()

    def get_completed_task_per_week(self, user):
        pass

    def get_avg_completion_time(self, user):
        pass

while True:
    print("\nMenu:")
    print("1. User Management")
    print("2. Task Management")
    print("3. Category Management")
    print("4. Task List")
    print("5. Analytics")
    print("6. Exit")

    try:
        choice = int(input("Enter your choice: "))

        if choice == 1:
            
            print("\nUser Management:")
            print("1. Add User")
            print("2. Update User Profile")
            print("3. View User Profiles")
            print("4. Back to Main Menu")

            user_choice = int(input("Enter your choice: "))

            if user_choice == 1:
                user = User()
                user.get_data()
            elif user_choice == 2:
                user = User()
                user.update_profile()
            elif user_choice == 3:
                user = User()
                user.get_profile()
            elif user_choice == 4:
                pass
            else:
                print("Invalid choice!")
        
        
        elif choice == 2:
            
            print("\nTask Management:")
            print("1. Add Task")
            print("2. Update Task")
            print("3. Mark Task as Completed")
            print("4. Delete Task")
            print("5. Back to Main Menu")

            task_choice = int(input("Enter your choice: "))

            if task_choice == 1:
                task = Task()
                task.add_task()
            elif task_choice == 2:
                task = Task()
                task.update_task()
            elif task_choice == 3:
                task = Task()
                task.mark_as_completed()
            elif task_choice == 4:
                task = Task()
                task.delete_task()
            elif task_choice == 5:
                pass
            else:
                print("Invalid choice!")

        elif choice == 3:
            
            print("\nCategory Management:")
            print("1. Take Category as input from the user")
            print("2. View Tasks by Category")
            print("3.Back to Main Menu" )

            category_choice = int(input("Enter your choice: "))

            if category_choice == 1:
                category = Category()
            elif category_choice == 2:
                category = Category()
                category.view_task_by_category()
            elif category_choice == 3:
                pass          
            else:
                print("Invalid choice!")

        elif choice == 4:

            print("\nTask List:")
            print("1. Filtered Task List")
            print("2. Display All Tasks")
            print("3. Back to Main Menu")

            task_list_choice = int(input("Enter your choice: "))

            task_list = TaskList()

            if task_list_choice == 1:
                task_list.filtered_task()
            elif task_list_choice == 2:
                task_list.display_tasks()
            elif task_list_choice == 3:
                pass
            else:
                print("Invalid choice!")

        elif choice == 5:
            
            print("\nAnalytics:")
            print("1. Get Completed Tasks Per Week")
            print("2. Get Average Completion Time")
            print("3. Back to Main Menu")

            analytics_choice = int(input("Enter your choice: "))

            analytics = Analytics()

            if analytics_choice == 1:
                user = User()
                analytics.get_completed_task_per_week(user)
            elif analytics_choice == 2:
                user = User()  # You can choose a user for which you want to get analytics
                analytics.get_avg_completion_time(user)
            elif analytics_choice == 3:
                pass
            else:
                print("Invalid choice!")

        elif choice == 6:
            break

        else:
            print("Invalid choice!")

    except ValueError:
        print("Invalid input. Please enter a number.")



Menu:
1. User Management
2. Task Management
3. Category Management
4. Task List
5. Analytics
6. Exit
Enter your choice: Ritika
Invalid input. Please enter a number.

Menu:
1. User Management
2. Task Management
3. Category Management
4. Task List
5. Analytics
6. Exit
Enter your choice: 1

User Management:
1. Add User
2. Update User Profile
3. View User Profiles
4. Back to Main Menu
Enter your choice: 1
Enter the username: Ritika
Enter the fullname: ritika jadhav
Enter the email: ritka123@gmail.com
User added successfully.

Menu:
1. User Management
2. Task Management
3. Category Management
4. Task List
5. Analytics
6. Exit
Enter your choice: 1

User Management:
1. Add User
2. Update User Profile
3. View User Profiles
4. Back to Main Menu
Enter your choice: 3
1 krishan krishan chavan krish111@gmail.com
2 kj kiran jamdhade kj555@gmail.com
3 Ajinkya ajinkya pawar Ap123@gmail.com
4 roshan roshan sawant roshansawant071@gmail.com
5 aashish aashish cyber sucess as123@gmail.com
6 kirti 1 KIRTIS